# Описание проекта:

Необходимо оценить список предложенных гипотез для увеличиния прибыли в крупном интернет-магазине. В процессе исследования будет проведен А/В тест, а также полученные результаты будут предоставлены отделу маркетинга для изучения.

# Описание данных
Данные для первой части
Датасет hypothesis.csv:
* Hypothesis — краткое описание гипотезы;
* Reach — охват пользователей по 10-балльной шкале;
* Impact — влияние на пользователей по 10-балльной шкале;
* Confidence — уверенность в гипотезе по 10-балльной шкале;
* Efforts — затраты ресурсов на проверку гипотезы по 10-балльной шкале. Чем больше значение Efforts, тем дороже проверка гипотезы.

Данные для второй части

Датасет orders.csv:
* transactionId — идентификатор заказа;
* visitorId — идентификатор пользователя, совершившего заказ;
* date — дата, когда был совершён заказ;
* revenue — выручка заказа;
* group — группа A/B-теста, в которую попал заказ.

Датасет visitors.csv:
* date — дата;
* group — группа A/B-теста;
* visitors — количество пользователей в указанную дату в указанной группе A/B-теста

# Предобработка данных

In [2]:
#загрузка библиотек и подготовках данных
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns
from pymystem3 import Mystem
from collections import Counter
from scipy import stats as st
import datetime as dt

In [3]:
#загрузим датасеты
try:
    hypothesis = pd.read_csv('hypothesis.csv')
    orders = pd.read_csv('orders.csv', sep = ',')
    visitors = pd.read_csv('visitors.csv', sep = ',')
except FileNotFoundError:
    hypothesis = pd.read_csv('/datasets/hypothesis.csv')
    orders = pd.read_csv('/datasets/orders.csv', sep = ',')
    visitors = pd.read_csv('/datasets/visitors.csv', sep = ',')

In [4]:
#просмотрим данные и сделаем предобработку
hypothesis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Hypothesis  9 non-null      object
 1   Reach       9 non-null      int64 
 2   Impact      9 non-null      int64 
 3   Confidence  9 non-null      int64 
 4   Efforts     9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 488.0+ bytes


In [5]:
hypothesis.head()

,Hypothesis,Reach,Impact,Confidence,Efforts
0,"Добавить два новых канала привлечения трафика,...",3,10,8,6
1,"Запустить собственную службу доставки, что сок...",2,5,4,10
2,Добавить блоки рекомендаций товаров на сайт ин...,8,3,7,3
3,"Изменить структура категорий, что увеличит кон...",8,3,3,8
4,"Изменить цвет фона главной страницы, чтобы уве...",3,1,1,1


In [6]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   transactionId  1197 non-null   int64 
 1   visitorId      1197 non-null   int64 
 2   date           1197 non-null   object
 3   revenue        1197 non-null   int64 
 4   group          1197 non-null   object
dtypes: int64(3), object(2)
memory usage: 46.9+ KB


In [10]:
orders.head()

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B


In [12]:
visitors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      62 non-null     object
 1   group     62 non-null     object
 2   visitors  62 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


In [11]:
visitors.head()

,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756


In [9]:
#Чтобы часть столбцов в дальнейшем не скрывалась, настроим принудительное отображение
pd.set_option('max_colwidth', 200)

In [13]:
#приведем к нижнему регистру названия столбцов в датасете hypothesis
hypothesis.columns = map(str.lower, hypothesis.columns)

In [14]:
#проверим названия столбцов
hypothesis.columns.to_list()

['hypothesis', 'reach', 'impact', 'confidence', 'efforts']

In [ ]:
# преобразуем данные о времени в датасете visitors
visitors['date'] = visitors['date'].map(
    lambda x: dt.datetime.strptime(x, '%Y-%m-%d')
)
# преобразуем данные о времени в датасете orders
orders['date'] = orders['date'].map(
    lambda x: dt.datetime.strptime(x, '%Y-%m-%d')
)

<div class="alert alert-success">

**✅Комментарий ревьюера**

Молодец, что изменил типы данных и привел названия колонок к одному формату</div>

Скорректировали названия столбцов(изменили регистр), изменили формат данных для дат. Дубликаты и пропуски не обнаружены. Можно переходить к анализу

### Приоритизация гипотез
**В полученном датасете 9 гипотез по увеличению выручки интернет-магазина с указанными параметрами Reach, Impact, Confidence, Effort**

**Применение фреймворка ICE для приоритизации гипотез. Сортировка их по убыванию приоритета.**

In [ ]:
hypothesis['ICE'] = (hypothesis['impact'] * hypothesis['confidence'] / hypothesis['efforts'])
hypothesis[['hypothesis', 'ICE']].sort_values(by = 'ICE', ascending=False).round(2)

Лидером получилась 8 гипотеза

**Применение фреймворка RICE для приоритизации гипотез. Сортировка их по убыванию приоритета.**

In [ ]:
hypothesis['RICE'] = (hypothesis['reach'] * hypothesis['impact'] * hypothesis['confidence'] / hypothesis['efforts'])
hypothesis[['hypothesis', 'RICE']].sort_values(by = 'RICE', ascending=False).round(2)

Лидером получилсь 7 гипотеза

**Как изменилась приоритизация гипотез при применении RICE вместо ICE. Почему так произошло?**

In [ ]:
#посмотримт на таблицу
hypothesis.head(10)

<div class="alert alert-success">

**✅Комментарий ревьюера**

Верно посчитал значения фреймворков. Согласен с объяснениями различий</div>

Пятерка лучших гипотез (0, 2, 6, 7, 8) осталась неизменной в методах ICE и RICE. Однако, при расчете RICE гипотеза 7 вышла на первое место, благодаря высокому показателю параметра охвата (Reach) равному 10. Это напоминает о необходимости учитывать количество пользователей, затронутых гипотезой, помимо ее силы приоритетности.

### Анализ А/В теста
**После проведения А/В теста мы получили резальтаты которые хранятся в файлах orders и visitors**
#### Построим график кумулятивной выручки по группам. Сделаем выводы и предположения.

In [ ]:
orders.head()

In [ ]:
visitors.head()

In [ ]:
#Проверим, есть ли пользователи, которые попали в обе группы
duplicates_users = np.intersect1d(orders.query('group == "A"')['visitorId'].unique(), orders.query('group == "B"')['visitorId'].unique())
duplicates_users

In [ ]:
#удалим пользователей, которые попали в обе группы
orders = orders[~orders['visitorId'].isin(duplicates_users)]

In [ ]:
#Проверяем
np.intersect1d(orders.query('group == "A"')['visitorId'].unique(), orders.query('group == "B"')['visitorId'].unique())

In [ ]:
#Соберем все данные в массив уникальных пар значений дат и групп теста
dates_groups = orders[['date','group']].drop_duplicates()

#далее, получаем агрегированные кумулятивные о заказах с разбивкой по дням
orders_aggregated = dates_groups.apply(lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])]. \
                                     agg({'date' : 'max', 
                                          'group' : 'max', 
                                          'transactionId' : pd.Series.nunique, 
                                          'visitorId' : pd.Series.nunique, 
                                          'revenue' : 'sum'}), axis=1).sort_values(by=['date','group'])

#выведем агрегированные кумулятивные данные о посетителях интернет-магазина с разбивкой по дням
visitors_aggregated = dates_groups.apply(lambda x: visitors[np.logical_and(visitors['date'] <= x['date'], visitors['group'] == x['group'])]. \
                                       agg({'date' : 'max', 
                                            'group' : 'max', 
                                            'visitors' : 'sum'}), axis=1).sort_values(by=['date','group'])

#также, объединим кумулятивные данные в одной таблице и присвоим столбцам понятные названия
cumulative_data = orders_aggregated.merge(visitors_aggregated, left_on=['date', 'group'], right_on=['date', 'group'])

#Переименуем столбцы
cumulative_data.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

cumulative_data.head(5)

In [ ]:
# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А
cumulativeRevenueA = cumulative_data[cumulative_data['group']=='A'][['date','revenue', 'orders']]
# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B
cumulativeRevenueB = cumulative_data[cumulative_data['group']=='B'][['date','revenue', 'orders']]

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.figure(figsize=(15,5))
#Строим график выручки группы А
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue'], label='К.выручка группы A')
#Строим график выручки группы B
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue'], label='К.выручка группы B')
plt.title('График кумулятивной выручки по группам\n  ')
plt.xlabel('\nДата')
plt.ylabel('Выручка\n')
plt.grid()
plt.legend();

Рост выручки в обеих группах примерно одинаков, а также в обеих группах происходит небольшой резкий скачок примерно в одно и то же время - 29 августа 2019 года. Стоит отметить этот момент для дальнейшего анализа.

#### Построим график кумулятивного среднего чека по группам. Сделаем выводы и предположения.

In [ ]:
plt.figure(figsize=(15,5))
plt.grid()
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue']/cumulativeRevenueA['orders'], label='A')
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue']/cumulativeRevenueB['orders'], label='B')
plt.title('График кумулятивного среднего чека по группам\n  ')
plt.xlabel('\nДата')
plt.ylabel('Сумма среднего чека\n')
plt.legend() ;

По графику видим, что значения среднего чека в обеих группах не являются стабильными и могут содержать выбросы, искажающие общую картину.

#### Постройте график относительного изменения кумулятивного среднего чека группы B к группе A. Сделайте выводы и предположения.

In [ ]:
# собираем данные в одном датафрейме
mergedCumulativeRevenue = cumulativeRevenueA.merge(cumulativeRevenueB, left_on='date', right_on='date', how='left', suffixes=['A', 'B'])

# cтроим отношение средних чеков
plt.figure(figsize=(15,5))
plt.grid()
plt.plot(mergedCumulativeRevenue['date'], (mergedCumulativeRevenue['revenueB']/mergedCumulativeRevenue['ordersB'])/(mergedCumulativeRevenue['revenueA']/mergedCumulativeRevenue['ordersA'])-1)
plt.title('График относительного изменения кумулятивного среднего чека группы B к группе A\n  ')
plt.xlabel('\nДата')
plt.ylabel('Изменение среденего чека \n')
# добавляем ось X
plt.axhline(y=0, color='black', linestyle='--');

Средние чеки в обеих группах не являются стабильными, так как могут содержать выбросы, искажающие общую картину. Это подтверждается графиком относительного различия в среднем чеке между группами, где видно значительное изменение значений в определенные даты, вероятно, из-за совершения аномальных заказов.

#### Постройте график кумулятивного среднего количества заказов на посетителя по группам. Сделайте выводы и предположения.

In [ ]:
# считаем кумулятивную конверсию
cumulative_data['conversion'] = cumulative_data['orders']/cumulative_data['visitors']

# отделяем данные по группе A
cumulativeDataA = cumulative_data[cumulative_data['group']=='A']

# отделяем данные по группе B
cumulativeDataB = cumulative_data[cumulative_data['group']=='B']

# строим графики
plt.figure(figsize=(15,5))
plt.grid()
plt.plot(cumulativeDataA['date'], cumulativeDataA['conversion'], label='A')
plt.plot(cumulativeDataB['date'], cumulativeDataB['conversion'], label='B')
plt.legend()
plt.title('График кумулятивной конверсии по группам\n  ')
plt.xlabel('\nДата')
plt.ylabel('Конверсия\n')
# задаем масштаб осей
plt.axis([dt.datetime(2019, 8, 1), dt.datetime(2019, 9, 1), 0, 0.05]);

Изначально конверсия в сегменте А была выше, однако позднее конверсия в сегменте В выровнялась и зафиксировалась на более высоком уровне, чем в сегменте А.

#### Построим график относительного изменения кумулятивного среднего количества заказов на посетителя группы B к группе A. Сделаем выводы и предположения.

In [ ]:
mergedCumulativeConversions = cumulativeDataA[['date','conversion']].merge(cumulativeDataB[['date','conversion']], left_on='date', right_on='date', how='left', suffixes=['A', 'B'])
plt.figure(figsize=(15,5))
plt.grid()
plt.plot(mergedCumulativeConversions['date'], mergedCumulativeConversions['conversionB']/mergedCumulativeConversions['conversionA']-1, label="Относительный прирост конверсии группы B относительно группы A")
plt.legend()

plt.axhline(y=0, color='black', linestyle='--')
plt.axhline(y=0.2, color='red', linestyle='--')
plt.axis(["2019-08-01", '2019-09-01', -0.4, 0.4])
plt.title('График относительного изменения кумулятивной конверсии группы B к группе A\n  ')
plt.xlabel('\nДата')
plt.ylabel('Относительное изменение конверсии\n');

Практически с самого начала группа В лидирует по конверсии и приняла стибальные +15% относительно группы А.

#### Постройте точечный график количества заказов по пользователям. Сделайте выводы и предположения.

In [ ]:
ordersByUsers = orders.drop(['group', 'revenue', 'date'], axis=1).groupby('visitorId', as_index=False). \
                                                                        agg({'transactionId' : pd.Series.nunique})

ordersByUsers.columns = ['user_id','orders']

ordersByUsers.sort_values(by='orders',ascending=False).head(10)

# серия из чисел от 0 до количества наблюдений в ordersByUsers
plt.figure(figsize=(15,5))
plt.grid()
plt.title('График количества заказов по пользователям\n  ')
plt.xlabel('\nКол-во пользователей')
plt.ylabel('Кол-во заказов\n')
x_values = pd.Series(range(0, len(ordersByUsers)))
plt.scatter(x_values, ordersByUsers['orders']);

Самое большое количество заказов - 1. Совершивших более 2 заказов гораздо меньше. Совершивших 3 заказа единицы. Возможно это и будет являться выбросом

#### Посчитаем 95-й и 99-й перцентили количества заказов на пользователя. Выберем границу для определения аномальных пользователей.

In [ ]:
print(np.percentile(ordersByUsers['orders'], [95, 99])) 

95% совершают не более 1 заказов. 99% совершают 2 заказа. Аномальным будем считать все, что более этого

#### Построим точечный график стоимостей заказов. Сделаем выводы и предположения.

In [ ]:
x_values = pd.Series(range(0,len(orders['revenue'])))
plt.figure(figsize=(15,5))
plt.grid()
plt.title('График стоимостей заказов\n ')
plt.xlabel('\nКол-во пользователей')
plt.ylabel('Стоимость заказа\n')
plt.scatter(x_values, orders['revenue'], alpha=0.3);

График имеет несколько выбросов, что затрудняет его анализ.

Для получения более точной картины, необходимо удалить явные выбросы в значениях 'revenue', превышающие 200000.

In [ ]:
#Посмотрим точные суммы аномальных заказов для выбора
orders['revenue'].sort_values().tail()

В данных обнаружены два аномальных заказа: один на сумму 1294500, а второй на 202740.

In [ ]:
#отберем заказы стоимость которых меньше 200000
norm_orders = orders[orders['revenue']<=200000]['revenue']
xn_values = pd.Series(range(0,len(norm_orders)))
plt.figure(figsize=(15,5))
plt.grid()
plt.title('График стоимостей заказов')
plt.xlabel('\nКол-во пользователей')
plt.ylabel('Стоимость заказа\n')
plt.scatter(xn_values, norm_orders, alpha=0.5);

Большинство заказов не превышают 200000, но были обнаружены два необычных заказа на суммы 1294500 и 202740. Чтобы получить более точное представление о распределении заказов, можно использовать перцентили.

#### Посчитаем 95-й и 99-й перцентили стоимости заказов. Выберем границу для определения аномальных заказов.

In [ ]:
np.percentile(orders['revenue'], [95, 99])

Не более чем у 95% заказов чек выше 26 785 рублей, и не более чем у 1% дороже 53 904 рублей. Границей определения аномальных заказов установим планку 53 904.

#### Посчитаем статистическую значимость различий в среднем количестве заказов на посетителя между группами по «сырым» данным. Сделаем выводы и предположения.

Подготовим данные. Соберем в общую талицу все необходимые данные по заказам и визитам пользователей, добавив при этом кумулятивные показатели

In [ ]:
orders.sample()

In [ ]:
visitors.sample()

In [ ]:
visitorsADaily = visitors[visitors['group']=='A'][['date', 'visitors']]
visitorsADaily.columns = ['date', 'visitorsPerDateA']

visitorsACummulative = visitorsADaily.apply(lambda x: visitorsADaily[visitorsADaily['date'] <= x['date']]. \
                                            agg({'date' : 'max', 'visitorsPerDateA' : 'sum'}), axis=1)
visitorsACummulative.columns = ['date', 'visitorsCummulativeA']

visitorsBDaily = visitors[visitors['group']=='B'][['date', 'visitors']]
visitorsBDaily.columns = ['date', 'visitorsPerDateB']

visitorsBCummulative = visitorsBDaily.apply(lambda x: visitorsBDaily[visitorsBDaily['date'] <= x['date']]. \
                                            agg({'date' : 'max', 'visitorsPerDateB' : 'sum'}), axis=1)
visitorsBCummulative.columns = ['date', 'visitorsCummulativeB']
ordersADaily = orders[orders['group']=='A'][['date', 'transactionId', 'visitorId', 'revenue']]\
    .groupby('date', as_index=False)\
    .agg({'transactionId' : pd.Series.nunique, 'revenue' : 'sum'})
ordersADaily.columns = ['date', 'ordersPerDateA', 'revenuePerDateA']

ordersACummulative = ordersADaily.apply(
    lambda x: \
        ordersADaily[ordersADaily['date'] <= x['date']]\
            .agg({
                'date' : 'max',
                'ordersPerDateA' : 'sum',
                'revenuePerDateA' : 'sum'}), axis=1).sort_values(by=['date'])
ordersACummulative.columns = ['date', 'ordersCummulativeA', 'revenueCummulativeA']

ordersBDaily = orders[orders['group']=='B'][['date', 'transactionId', 'visitorId', 'revenue']]\
    .groupby('date', as_index=False)\
    .agg({'transactionId' : pd.Series.nunique, 'revenue' : 'sum'})
ordersBDaily.columns = ['date', 'ordersPerDateB', 'revenuePerDateB']

ordersBCummulative = ordersBDaily.apply(
    lambda x: \
        ordersBDaily[ordersBDaily['date'] <= x['date']]\
            .agg({
                'date' : 'max',
                'ordersPerDateB' : 'sum',
                'revenuePerDateB' : 'sum'}), axis=1).sort_values(by=['date'])
ordersBCummulative.columns = ['date', 'ordersCummulativeB', 'revenueCummulativeB']
data = ordersADaily.merge(ordersBDaily, left_on='date', right_on='date', how='left')\
    .merge(ordersACummulative, left_on='date', right_on='date', how='left')\
    .merge(ordersBCummulative, left_on='date', right_on='date', how='left')\
    .merge(visitorsADaily, left_on='date', right_on='date', how='left')\
    .merge(visitorsBDaily, left_on='date', right_on='date', how='left')\
    .merge(visitorsACummulative, left_on='date', right_on='date', how='left')\
    .merge(visitorsBCummulative, left_on='date', right_on='date', how='left')
data.head()

Теперь имеем возможность посчитать статистическую значимость различия в конверсии между группами. Создаем переменные  ordersByUsersA и ordersByUsersB со столбцами userId, ordersId

In [ ]:
ordersByUsersA = orders[orders['group']=='A'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsersA.columns = ['userId', 'orders']

ordersByUsersB = orders[orders['group']=='B'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsersB.columns = ['userId', 'orders']

Для проведения проверки критерием Манна-Уитни необходимо создать переменные sampleA и sampleB, в которых будет содержаться количество заказов, сделанных пользователями из разных групп. Если пользователь не сделал ни одного заказа, то его значение будет равно нулю.

In [ ]:
sampleA = pd.concat([ordersByUsersA['orders'],pd.Series(0, index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])), name='orders')],axis=0)

sampleB = pd.concat([ordersByUsersB['orders'],pd.Series(0, index=np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])), name='orders')],axis=0)

* Нулевая гипотеза H0 - статистически значимых различий в конверсиях между группами нет
* Альтернативная гипотеза H1 - статистически значимые различия есть

In [ ]:
#Применим критерий и отформатируем p-value, округлив его до трёх знаков после запятой
alpha = 0.05
print("p-value = {0:.3f}".format(st.mannwhitneyu(sampleA, sampleB, alternative = 'two-sided')[1]))
print("относительный прирост конверсии группы B = {0:.3f}".format(sampleB.mean()/sampleA.mean()-1)) 

Мы применили критерий Манна-Уитни для проверки гипотезы о равенстве конверсии в двух группах. Полученное значение p-value равно 0.011, что больше заданного уровня значимости alpha = 0.05. Следовательно, мы отвергаем нулевую гипотезу о равенстве конверсии между группами. Относительный прирост конверсии группы В к А равен 16%.

#### Посчитаем статистическую значимость различий в среднем чеке заказа между группами по «сырым» данным. Сделаем выводы и предположения.

* Нулевая гипотеза H0 - статистически значимых различий в среднем чеке между группами нет
* Альтернативная гипотеза H1 - статистически значимые различия есть

In [ ]:
alpha = 0.05
print("p-value = {0:.3f}".format(st.mannwhitneyu(orders[orders['group']=='A']['revenue'], orders[orders['group']=='B']['revenue'], alternative = 'two-sided')[1]))
print("Относительный показатель 'B' и 'A' = {0:.3f}".format(orders[orders['group']=='B']['revenue'].mean()/orders[orders['group']=='A']['revenue'].mean()-1)) 

P-value равный 0.829 говорит о том, что вероятность получить различия между группами случайно составляет 82.9%. Это говорит о том, что статистически значимых различий между группами нет. По "сырым" данным чеки похожи.

Относительный прирост конверсии группы B на 28.7% может свидетельствовать о том, что внедренное изменение на сайте (если такое было) положительно повлияло на конверсию. Однако, для более точной оценки необходимо также учитывать другие факторы, которые могут влиять на конверсию.

#### Посчитаем статистическую значимость различий в среднем количестве заказов на посетителя между группами по «очищенным» данным. Сделаем выводы и предположения.

Подготовим данные, очищенные от аномалий.
В прошлых вычислениях мы определили пользователей как аномальных, если они:
* совершили 3 и более заказов
* заказали товары на сумму более 53 904 рублей

Сделаем срезы пользователей:

* с числом заказов больше 2 — usersWithManyOrders
* совершивших заказы дороже 53904 — usersWithExpensiveOrders.

Посмотрим на количество аномлаьных пользователей при помощи метода shape().
Тут нужно отобрать свои данные и подставить в код

In [ ]:
usersWithManyOrders = pd.concat([ordersByUsersA[ordersByUsersA['orders'] > 2]['userId'], ordersByUsersB[ordersByUsersB['orders'] > 2]['userId']], axis = 0)
usersWithExpensiveOrders = orders[orders['revenue'] > 53904]['visitorId']
abnormalUsers = pd.concat([usersWithManyOrders, usersWithExpensiveOrders], axis = 0).drop_duplicates().sort_values()
display(abnormalUsers.head(5))
print("Всего аномальных пользователей =" ,len(abnormalUsers)) 

Всего 16 аномальных пользователей. Посмотрим, как изменятся расчет без их участия

In [ ]:
sampleAFiltered = pd.concat([ordersByUsersA[np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))]['orders'], \
                             pd.Series(0, index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])), \
                                       name='orders')],axis=0)

sampleBFiltered = pd.concat([ordersByUsersB[np.logical_not(ordersByUsersB['userId'].isin(abnormalUsers))]['orders'], \
                             pd.Series(0, index=np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])), \
                                       name='orders')],axis=0) 

* Нулевая гипотеза H0 - статистически значимых различий в конверсиях между группами нет
* Альтернативная гипотеза H1 - статистически значимые различия есть

In [ ]:
#Применим статистический критерий Манна-Уитни к полученным выборкам
alpha = 0.05
print("p-value = {0:.3f}".format(st.mannwhitneyu(sampleAFiltered, sampleBFiltered, alternative = 'two-sided')[1]))
print("Относительный показатель 'B' и 'A' = {0:.3f}".format(sampleBFiltered.mean()/sampleAFiltered.mean()-1)) 

Уровень значимости (p-value) равен 0.013, что означает, что что статистически значимых различий в конверсии между группами нет Следовательно, мы отвергаем нулевую гипотезу. Это значит: по «очищенным» данным в конверсии групп A и B есть статистически значимые различия. А относительный выигрыш группы "B" над "A" равен почти 19% (выше, чем с «сырыми» данными - 16%)

#### Посчитаем статистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным. Сделаем выводы и предположения.

* Нулевая гипотеза H0 - статистически значимых различий в среднем чеке между группами нет
* Альтернативная гипотеза H1 - статистически значимые различия есть

In [ ]:
print("alpha =",0.05)
print("p-value = {0:.3f}".format(st.mannwhitneyu(
    orders[np.logical_and(
        orders['group']=='A',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'],
    orders[np.logical_and(
        orders['group']=='B',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'], alternative = 'two-sided')[1]))

print("Относительный показатель В и А = {0:.3f}".format(
    orders[np.logical_and(orders['group']=='B',np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'].mean()/
    orders[np.logical_and(
        orders['group']=='A',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'].mean() - 1)) 

Уровень значимости (p-value) равен 0.788, и поэтому мы можем считать, что в среднем чеке нет различий(как и с "сырыми" данными). Но вот средй чек группы В оказался незначительно(3%) ниже среднего чека группы А, в то время как по сырым данным, он был почти на 29% пбольше. Вот таким образом ощутимо повлияли на результаты наши аномальные заказы с большими суммами.

#### Примите решение по результатам теста и объясните его. Варианты решений:
1. Остановить тест, зафиксировать победу одной из групп.
2. Остановить тест, зафиксировать отсутствие различий между группами.
3. Продолжить тест.

После анализа данных, мы узнали, что группа В показала повышение конверсии на 19% в "очищенных" данных и на 16% в "сырых" данных. Средние чеки групп примерно равны в "очищенных данных". Рекомендуем закончить А/В тест и закрепить победу группы В по конверсии. В дальнейшем, отдел маркетинга может приступить к стимулированию увеличения среднего чека.